### GAN

GAN的核心是生成器G和验证器D的对抗，通过不断对抗，让G和D都优化

![gan_image_intro](../images/gan_image_intro.png)

从公式来看，就是D(x)需要不断增大，G(z)也就是1-D(x)要不断减小。公式中D即是计算两个分布相似度的方法。

![](../images/gan_train.png)

### 如何训练

需要先固定生成器，持续优化鉴定器；然后固定鉴定器，持续优化生成器。这里是指每一个epoch，而不是分为完全独立的两个网络，参考[dcgan_practice](./dcgan_practice.ipynb) training部分

![gan_how_to_train](../images/gan_how_to_train.png)

鉴定器的训练就是卷积神经网络的训练，梯度计算比较常规。关键在于loss函数，不同的Gan模型不同。

![gan_d_math](../images/gan_d_math.png)

生成器的训练，总的来说是通过计算鉴定器和生成器的分布相似度来进行反馈的。

基于JS Divergence做相似度计算,这里注意其和KL Divergence的差别在于，JS是对称的。当$P_r$和$P_g$相等时，可以得到最好的训练结果。

![js_divergence](../images/js_divergence.png)

![js_divergence_train_g](../images/js_divergence_train_g.png)

GAN的模型很多，可以参见[这里](https://github.com/hindupuravinash/the-gan-zoo), 其中核心思想主要是对鉴定器训练卷积核以及生成器分布相似度计算的优化。注意，这里所说的分布相似度算法是用来证明GAN的loss函数用的，loss函数中并不需要直接使用。

### DCGAN（DeConverlutionGAN）

DCGAN的优势在于，鉴定器训练采用了加padding与加Stride的方式升维，可以生成大图片。生成器仍然采用JS Divergence进行训练。

![wcgan](../images/wcgan.png)

![dcgan_discriminator](../images/dcgan_discriminator.png)

![dcgan_kernel](../images/dcgan_kernel.png)

JS Divergence和KL divergence有一个很大问题, 当两个分布完全不相关时(这在GAN训练初期很常见，需要调整出一个非常好的初始参数才能解决)，相似度得到是相同的常量，这会导致无法训练（因为梯度不变化）。详情参考：[blog](https://medium.com/@jonathan_hui/gan-wasserstein-gan-wgan-gp-6a1a2aa1b490)

![divergence_problem](../images/divergence_problem.png)



### WGAN(Wasserstein Distance GAN)

WGAN的优势在于更换了计算分布相似度的方法，采用了Wasserstein Distance。其核心思想是计算通过分别采样，计算两个分布的距离差，并通过最小化移动策略，让两个分布的距离缩小。

![wasserstein_distance_cost_plan](../images/wasserstein_distance_cost_plan.png)

下图找到最小化策略，针对离散情况，动态规划

![wassertein_distance_min_plan](../images/wassertein_distance_min_plan.png)

下图针对连续情况，公式。f要求1-Lipschitz，即导数一直小于1。论文中通过weight-clipping的方式实现的1-Lipschitz，看下文。

![wasserstein_distance_math](../images/wasserstein_distance_math.png)

### weight-clipping

通过限制w的取值范围，来尝试实现，但效果不好，比较暴力，这样会限制模型的表达能力, 因为这和Regularization是类似的,参考[overfit_underfit](../overfit_underfit.ipynb)中的Regularization节。

![weight-clipping](../images/weight-clipping.png)



### WGAN-GP(Wasserstein Distance GAN - Gradient Penalty)

WGAN-GP通过梯度惩罚，实现了上文的1-Lipschitz。下图中，右侧红线是解释$\hat{x}$, 它是通过对real和generate的图片的插值采样得来的。

![wgan-gp](../images/wgan-gp.png)


### 经典GAN Performance对比

![gans_vs](../images/gans_vs.png)